# Mini-projet - Pour aller plus loin
# Récupérer des méta-données EXIF de plusieurs Images pour les placer sur une même carte

Pour commencer on charge les modules ...

In [ ]:
import sys
!{sys.executable} -m pip install pillow folium

... puis on les importe.

In [ ]:
from PIL import Image
import folium

## Récupération des coordonnées terrestres

In [ ]:
def coord(meta) :
    # Récupération de la localisation terrestre
    gps = meta[34853]

    # gps est à nouveau un dictionnaire
    lat_dir = gps[1] #N ou S pour North ou South
    lat = gps[2]
    lon_dir = gps[3] #W ou E pour West ou East
    lon = gps[4]

    # Conversion décimale de la latitude et de la longitude
    latitude = lat[0] + lat[1]/60.+ lat[2]/3600.
    longitude = lon[0] + lon[1]/60. + lon[2]/3600.

    #On change le signe de la latitude si besoin
    if lat_dir == 'S':
        latitude = -1*latitude

    #On change le signe de la longitude si besoin
    if lon_dir == 'W':
        longitude = -1*longitude

    return latitude, longitude

## Récupération de la date de prise de vue

In [ ]:
def temps(meta) :
    # Récupération de la date de prise de vue
    date = meta[36867]
    annee, mois, jourHeure, minute, seconde = date.split(':')
    jour, heure = jourHeure.split(' ')

    return annee, mois, jour, heure, minute, seconde

## Récupération de l'orientation de la prise de vue 
Pour insérer la photo sur la carte, on a besoin de savoir si la photo a été prise en mode paysage ou portrait. On effectude si besoin une rotation pour mettre la photo dans le bon sens.

In [ ]:
def reorientation(meta) :    
    # Récupération de l’orientation
    orientation = info [274]
    if orientation == 6  or orientation == 8 :
        # format portrait
        formatIcon = (48 ,86)
    else :
        # format paysage
         formatIcon = (86 ,48)
    return formatIcon

## Le programme principal

In [ ]:
######################################################################
### récupération de toutes les informations pour toutes les photos ###
######################################################################
nom = input ( " Quel est votre nom ? " )
prenom = input ( " Quel est votre prénom ? " )
nbPhotos = int ( input ( " Combien de photos avez−vous ? " ) )

# pour le calcul du centre de la carte
# on va calculer les coordonnées du point moyen
xMoy, yMoy = 0, 0

# pour stocker toutes les coord de toutes les photos
ptPhotos = []
# pour stocker toutes les formats pour les icônes
forIcones = [] 
# pour stocker les textes concernant les prises de vues
textDates = []

for i in range(nbPhotos):
    # ouverture de la photo
    photo = Image.open("chouchou"+str(i)+".jpg")
    # récupération des données
    info = photo._getexif()
    
    # coordonnées
    latitude, longitude = coord(info)
    ptPhotos.append([latitude, longitude])
    #pour le calcul du point moyen
    xMoy = xMoy + latitude
    yMoy = yMoy + longitude
    
    #format des icônes
    formatIcone = reorientation(info) 
    forIcones.append(formatIcone)
    
    #texte pour les dates de prise de vue
    annee, mois, jour, heure, minute, seconde = temps(info)
    textDates.append(nom+" "+prenom+" \n à : "+heure+"h "+minute+" min \n l e "+jour+" / "+mois+" / "+annee)

#finalisation du calcul du point moyen
xMoy = xMoy/ nbPhotos
yMoy = yMoy/ nbPhotos


######################################################################
###                     création de la carte                       ###
######################################################################
# création de la carte centrée
carte =folium.Map(
    # carte centrée sur le point moyen
    location = ([xMoy , yMoy ]),
    #zoom de 16
    zoom_start = 16)

# placement de tous les marqueurs avec les photos/icons
for i in range(nbPhotos):
    folium.Marker(
        # coordonnées de la photo numéro i
        ptPhotos[i],
        # texte pour la date de prise de vue
        popup = textDates[ i ],
        # création d’un icône avec la photo
        icon = folium.CustomIcon("chouchou"+str(i)+".jpg", icon_size = forIcones[i])
        ).add_to(carte)

# ajout d'un icône pour le point moyen
folium.Marker(
    [xMoy, yMoy], 
    popup = "point moyen",
    icon = folium.Icon(color = 'red', icon = 'info−sign')
    ).add_to(carte)

# ajout du parcours à vol d’oiseau
folium.PolyLine(
    ptPhotos ,
    color = "red",
    weight = 2.5,
    opacity = 1
    ).add_to(carte)

########################################################################"
carte.save('carte_chouchou.html')